# RAG Pipeline rag-checker


In [1]:
from pathlib import Path
from dotenv import load_dotenv
import sys, pandas as pd

ROOT = Path.cwd().parent  # 노트북은 tests/ 하위에 있다고 가정
sys.path.append(str(ROOT))

from services.orchestrator import router_node
from ragas import SingleTurnSample

load_dotenv(ROOT / ".env")
print(f"ROOT - {ROOT}")
print("환경 변수 로드 완료")

2025-08-04 22:17:28,613 | INFO | Loading faiss with AVX2 support.
2025-08-04 22:17:28,760 | INFO | Successfully loaded faiss with AVX2 support.
2025-08-04 22:17:28,771 | INFO | Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.


ROOT - c:\Users\insung\Finance_Agent
환경 변수 로드 완료


In [ ]:
import litellm
from ragchecker import RAGChecker
from kiwipiepy import Kiwi

kiwi = Kiwi() 

def openai_api_function(prompts: list[str]) -> list[str]:
    """
    OpenAI의 gpt-4o 모델을 호출하고 응답 텍스트 리스트를 반환하는 커스텀 함수.
    """
    try:
        # litellm을 사용하여 OpenAI 모델을 배치로 호출합니다.
        response = litellm.batch_completion(
            model="gpt-4o",  # 사용할 OpenAI 모델 지정
            messages=[[{"role": "user", "content": p}] for p in prompts]
        )
        
        return [res.choices[0].message.content for res in response]
    except Exception as e:
        print(f"API 호출 중 에러 발생: {e}")
        # 에러 발생 시, 각 프롬프트에 대해 빈 문자열을 반환하여 평가가 중단되지 않게 함
        return ["" for _ in prompts]

class KiwiTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return [token.form for token in kiwi.tokenize(text)]
    def lemmatize(self, text):
        return [token.form for token in kiwi.tokenize(text)]

checker = RAGChecker(
    tokenizer=KiwiTokenizer(),
    language="ko",
    custom_llm_api_func=openai_api_function
)

print("rag-checker 초기화 완료")


W0804 22:17:53.102000 15316 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


rag-checker 초기화 완료


In [3]:
# 샘플 질문 정의
SAMPLE_QUESTIONS = [
    "OTP 비밀번호 오류 해제 방법 알려줘",
    "첫급여 우리적금에서 우대이율을 받기 위한 조건은 무엇인가요?",
    "정기적금을 만기 지난 뒤 해지하면 어떤 만기후이율이 적용되나요?",
    "오늘 날씨 어때?",
]

In [4]:
# 파이프라인 호출 → 평가 입력 변환
from services.orchestrator import router_node

records = []
for q in SAMPLE_QUESTIONS:
    res  = router_node.invoke(q)
    ctxs = res.get("context", "").split("\n\n") if res.get("context") else []
    records.append({"question": q, "answer": res["answer"], "contexts": ctxs})
print(f"{len(records)}개 레코드 수집 완료")


4개 레코드 수집 완료


## RAGchecker Evaluavtion

In [15]:
from types import SimpleNamespace
from ragchecker import RAGChecker, RAGResult, RAGResults
from ragchecker.metrics import all_metrics

rag_result_objects = []
for i, record in enumerate(records):
    structured_contexts = [
        SimpleNamespace(doc_id=f"doc_{i}_{j}", text=ctx) 
        for j, ctx in enumerate(record["contexts"])
    ]

    rag_result_objects.append(
        RAGResult(
            query_id=f"q_{i}",
            query=record["question"],
            response=record["answer"],
            retrieved_context=structured_contexts,
            gt_answer=None              # gt_answer는 실제 정답이 있을 경우 제공, 없으면 None
        )
    )

rag_results = RAGResults(results=rag_result_objects)

In [16]:
# 정답 텍스트가 없는 경우
print("\n### 1. Faithfulness (Generator) Evaluation ###")
metrics_to_run = ["faithfulness"]
overall = checker.evaluate(rag_results, metrics=metrics_to_run)
for k, v in overall.items():
    print(f"{k}: {v}")


print("\n--- 평가 완료 ---")
print("Faithfulness Score (0-100점, 높을수록 좋음):")
display(pd.DataFrame([overall]))

2025-08-04 22:36:40.465 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for response of 4 RAG results.



### 1. Faithfulness (Generator) Evaluation ###


  0%|          | 0/1 [00:00<?, ?it/s]22:36:40 - LiteLLM:INFO: utils.py:3260 - 
LiteLLM completion() model= gpt-4o; provider = openai
2025-08-04 22:36:40,472 | INFO | 
LiteLLM completion() model= gpt-4o; provider = openai
22:36:40 - LiteLLM:INFO: utils.py:3260 - 
LiteLLM completion() model= gpt-4o; provider = openai
2025-08-04 22:36:40,479 | INFO | 
LiteLLM completion() model= gpt-4o; provider = openai
22:36:40 - LiteLLM:INFO: utils.py:3260 - 
LiteLLM completion() model= gpt-4o; provider = openai
2025-08-04 22:36:40,484 | INFO | 
LiteLLM completion() model= gpt-4o; provider = openai
22:36:40 - LiteLLM:INFO: utils.py:3260 - 
LiteLLM completion() model= gpt-4o; provider = openai
2025-08-04 22:36:40,492 | INFO | 
LiteLLM completion() model= gpt-4o; provider = openai
22:36:41 - LiteLLM:INFO: utils.py:1262 - Wrapper: Completed Call, calling success_handler
2025-08-04 22:36:41,695 | INFO | Wrapper: Completed Call, calling success_handler
22:36:42 - LiteLLM:INFO: utils.py:1262 - Wrapper: Compl

overall_metrics: {}
retriever_metrics: {}
generator_metrics: {'faithfulness': np.float64(25.0)}

--- 평가 완료 ---
Faithfulness Score (0-100점, 높을수록 좋음):


,overall_metrics,retriever_metrics,generator_metrics
0,{},{},{'faithfulness': 25.0}


현재는 정답을 따로 구축하지 않아서 위의 코드블럭에서와 같이 'faithfulness'만 평가 함. \
이는 모델이 생성한 답변이 검색된 문맥에 얼마나 충실하게 근거하고 있는지를 평가하는 것으로 환각을 측정하는 것임.
\
\
정답 텍스트를 만들었을 경우 아래의 코드 청크에 올바르게 입력한 뒤 위에서 확인하지 못 했던 overall_metrics, retriever_merics를 평가할 수 있음.


```bash
# 정답 텍스트가 있는 경우

import pandas as pd
from types import SimpleNamespace
from ragchecker import RAGChecker, RAGResult, RAGResults
from ragchecker.metrics import all_metrics # 모든 지표를 임포트

# --- 데이터 준비 (gt_answer에 실제 정답 추가) ---
# 실제 평가를 위해서는 여기에 '모범 정답'과 '정답 문서 ID'를 넣어야함.
# 아래는 예시 데이터.
ground_truth_data = [
    {"answer": "우리WON뱅킹에서 비대면 실명확인을 통해 해제할 수 있습니다. 인터넷뱅킹에서는 불가능합니다.", "context_ids": ["doc_0_0"]}, # 가상의 정답
    {"answer": "첫급여 수령, 마케팅 동의 등의 조건을 충족하면 최대 연 4%의 우대금리를 받을 수 있습니다.", "context_ids": ["doc_1_x"]}, # 가상의 정답
    {"answer": "만기 후 기간에 따라 약정된 만기후이율이 적용되며, 보통 만기 시점보다 낮은 금리입니다.", "context_ids": ["doc_2_y"]}, # 가상의 정답
    {"answer": "날씨 정보는 제공하지 않습니다.", "context_ids": []} # 가상의 정답
]

rag_result_objects = []
for i, record in enumerate(records):
    structured_contexts = [
        SimpleNamespace(doc_id=f"doc_{i}_{j}", text=ctx) 
        for j, ctx in enumerate(record["contexts"])
    ]
    rag_result_objects.append(
        RAGResult(
            query_id=f"q_{i}",
            query=record["question"],
            response=record["answer"],
            retrieved_context=structured_contexts,
            gt_answer=ground_truth_data[i]["answer"], # 실제 정답 텍스트 제공
            gt_context_ids=ground_truth_data[i]["context_ids"] # 실제 정답 문서 ID 제공
        )
    )
rag_results = RAGResults(results=rag_result_objects)

# --- 평가 실행 ---
print("\n### 정답(GT) 포함 전체 파이프라인 평가 시작 ###")
try:
    # 'all_metrics'를 사용하여 모든 지표를 한 번에 평가
    checker.evaluate(rag_results, metrics=all_metrics)

    print("\n--- 평가 완료 ---")
    # 평가 결과는 rag_results.metrics 속성에 저장됩니다.
    
    print("\n### 종합 평가 (Overall Metrics) ###")
    display(pd.DataFrame([rag_results.metrics["overall_metrics"]]))

    print("\n### 검색기 평가 (Retriever Metrics) ###")
    display(pd.DataFrame([rag_results.metrics["retriever_metrics"]]))
    
    print("\n### 생성기 평가 (Generator Metrics) ###")
    display(pd.DataFrame([rag_results.metrics["generator_metrics"]]))

except Exception as e:
    print(f"평가 중 에러 발생: {e}")
```